# Chap04 分组

In [1]:
import numpy as np
import pandas as pd

## 分组模式及其对象
### 分组的一般模式
1. 明确分组的三个要素：**分组依据、数据来源、操作及其返回结果**
   - `df.groupby(分组依据)[数据来源].使用操作`
### 分组依据的本质
1. groupby的分组依据除了可以直接从列中按照名字获取，也可以通过一定的复杂逻辑分组
### groupby对象
1. 通过`gb.ngroups`属性，得到分组个数
2. 通过`gb.groups.keys()`属性，可以返回从组名映射到组索引列表的字典
3. 通过`gb.size()`属性，统计每个组的元素个数
4. 通过`gb.get_group(组名元组)`方法可以直接获取所在组对应行，此时必须知道组的具体名字
### 分组的三大操作
1. 聚合`agg` eg.依据性别分组，统计全国人口寿命的平均值
2. 变换`transform` eg.依据季节分组，对每一个季节的温度进行组内标准化
3. 过滤`filter` eg.依据班级筛选出组内数学分数的平均值超过80分的班级

In [2]:
df = pd.read_csv('./data/learn_pandas.csv')

In [5]:
condition = df.Weight > df.Weight.mean()
df.groupby(condition)['Height'].mean()

Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64

In [11]:
item = np.random.choice(list('abc'), df.shape[0])
df.groupby(item)['Height'].mean()

a    164.244643
b    162.135484
c    163.366154
Name: Height, dtype: float64

## 聚合函数
### 内置聚合函数
1. max/min/mean/median/count/all/any/idxmax/idxmin/mad/nunique/skew/quantile/sum/std/var/sem/size/prod
2. 性能相对较好
3. 缺点
   - 无法同时使用多个函数
   - 无法对特定的列使用特定的聚合函数
   - 无法使用自定义的聚合函数
   - 无法直接对结果的列名在聚合前进行自定义命名
### agg方法
1. 使用多个函数`gb.agg(聚合函数列表)`
2. 对特定的列使用特定的聚合函数`gb.agg(字典{列名:聚合函数or聚合函数列表})`
3. 在agg中可以使用具体的自定义函数
   - 由于传入的是序列，因此序列上的方法和属性都可以在函数中使用，只需保证返回值是标量即可
4. 聚合结果重命名，将函数的位置改写成元组`(新名字,函数原名)`
   - 对**一个或者多个列***使用**单个聚合**的时候，重命名需要加方括号`gb.agg([('my_sum', 'sum)])`

In [3]:
gb = df.groupby('Gender')[['Height','Weight']]
gb.agg(lambda x: x.mean()-x.min())

,Height,Weight
Gender,,
Female,13.79697,13.918519
Male,17.92549,21.759259


In [15]:
def my_func(s):
    res = 'High'
    if s.mean() <= df[s.name].mean():
        res = 'Low'
    return res
gb.agg(my_func)

,Height,Weight
Gender,,
Female,Low,Low
Male,High,High


In [16]:
gb.agg([('range', lambda x:x.max()-x.min()), ('my_sum','sum')])

Height          Weight        
        range   my_sum  range  my_sum
Gender                               
Female   24.8  21014.0   29.0  6469.0
Male     38.2   8854.9   38.0  3929.0

## 变换和过滤
### 变换函数与transform方法
1. 返回值为同长度的序列
   - 常用如累计函数`cumcount/cumsum/cumprod/cummax/cummin`
2. 当用自定义变换时需要使用`transform`方法，被调用的自定义函数，其传入值为**数据源的序列**，其最后的返回结果是**行列索引与数据源一致的DataFrame**
3. 事实上`transform`返回的是一个标量，返回同长度的序列是因为结果被广播到其所在的整个组
### 组索引与过滤
1. 在`groupby`对象中，定义了`filter`方法进行**组的筛选**，其中自定义函数的输入参数为数据源构成的`DataFrame`本身，同时只需保证自定义函数的返回为布尔值即可

In [4]:
gb.cummax()

,Height,Weight
0,158.9,46.0
1,166.5,70.0
2,188.9,89.0
3,NaN,46.0
4,188.9,89.0
...,...,...
195,170.2,63.0
196,170.2,63.0
197,170.2,63.0
198,193.9,89.0


In [5]:
gb.transform(lambda x: (x-x.mean())/x.std())

,Height,Weight
0,-0.058760,-0.354888
1,-1.010925,-0.355000
2,2.167063,2.089498
3,NaN,-1.279789
4,0.053133,0.159631
...,...,...
195,-1.048078,-0.354888
196,0.336968,0.385033
197,-1.048078,-0.539868
198,0.237570,-0.226342


In [6]:
# 标量广播
gb.transform('mean')

,Height,Weight
0,159.19697,47.918519
1,173.62549,72.759259
2,173.62549,72.759259
3,159.19697,47.918519
4,173.62549,72.759259
...,...,...
195,159.19697,47.918519
196,159.19697,47.918519
197,159.19697,47.918519
198,173.62549,72.759259


In [7]:
# 过滤得到所有容量大于100的组
gb.filter(lambda x: x.shape[0] > 100)

,Height,Weight
0,158.9,46.0
3,NaN,41.0
5,158.0,51.0
6,162.5,52.0
7,161.9,50.0
...,...,...
191,166.6,54.0
194,160.3,49.0
195,153.9,46.0
196,160.9,50.0


## 跨列分组
### apply的引入
1. 主要解决**多列数据同时处理**
### apply的使用
1. 自定义函数传入参数与filter完全一致，但返回值允许标量、一维Series和二维DataFrame
   - 标量：结果得到的是Series，索引与agg的结果一致
   - Series：得到的是DataFrame，行索引与标量情况一致，列索引为Series的索引
   - DataFrame：行索引最内层在每个组原先agg的结果索引上，再加一层返回的DataFrame行索引，同时分组结果DataFrame的列索引和返回的DataFrame列索引一致

In [8]:
def BMI(x):
    Height = x['Height']/100
    Weight = x['Weight']
    BMI_value = Weight/Height**2
    return BMI_value.mean()
gb.apply(BMI)

Gender
Female    18.860930
Male      24.318654
dtype: float64

In [9]:
# 返回标量
gb = df.groupby(['Gender','Test_Number'])[['Height','Weight']]
gb.apply(lambda x:0)

Gender  Test_Number
Female  1              0
        2              0
        3              0
Male    1              0
        2              0
        3              0
dtype: int64

In [10]:
gb.apply(lambda x: [0,0])

Gender  Test_Number
Female  1              [0, 0]
        2              [0, 0]
        3              [0, 0]
Male    1              [0, 0]
        2              [0, 0]
        3              [0, 0]
dtype: object

In [11]:
gb.apply(lambda x:pd.Series([0,0], index = ['a','b']))

a  b
Gender Test_Number      
Female 1            0  0
       2            0  0
       3            0  0
Male   1            0  0
       2            0  0
       3            0  0

In [14]:
gb.apply(lambda x:pd.DataFrame(np.ones((2,2)), index = ['a','b'], columns = pd.Index([('w','x'),('y','z')])))

w    y
                        x    z
Gender Test_Number            
Female 1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0
Male   1           a  1.0  1.0
                   b  1.0  1.0
       2           a  1.0  1.0
                   b  1.0  1.0
       3           a  1.0  1.0
                   b  1.0  1.0